In [1]:
# env oc pour OpenClassrooms
# context : Projet Nutriscore - "Open Food Facts"

# Partie 1 - Jupiter Notebook
# Partie 2 - Page web: Application
# Partie 3 - Présentation du sujet (diaporama)s

# __auteur__ = "Benjamin Schoenmaeker"
# __date__ =  "07-02-2023"

<h3>Calcul d'un nouveau Nutri-Score noté de 0 à 100</h3>

Ce code définit une interface graphique pour rechercher des informations sur un produit alimentaire.

Lorsque l'utilisateur saisit un nom de produit et clique sur le bouton "Rechercher", le code effectue une recherche dans une base de données de produits alimentaires. 
Le résultat de la recherche est filtré en fonction du nom du produit saisi et affiché sous forme de graphiques.

L'application publie les informations nutritionnelles sur ces produits (sucres, graisses, hydrates de carbone, protéines, sel et score nutritionnel).
La quantité d'informations affichées dépend du nombre de produits trouvés.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import VBox, HBox

import seaborn as sns
import plotly.express as px

# Exportation graphique des données
import os
import dataframe_image as dfi

from IPython.display import display

In [3]:
# Chargement des données dans un dataframe
open_food_dataframe = pd.read_csv('.\export\csv\sampling_open_food_job.csv', sep=';', low_memory=False) 

In [14]:
# Définir uune sortie pour le widget
out = widgets.Output()

# Définition de la fonction "search" qui effectue la recherche des produits alimentaires.
def search(product_name):
    e = 0
    out.clear_output()
    
    # Création d'un masque pour filtrer les données en fonction du nom de produit saisi.
    mask = open_food_dataframe['product_name'].str.contains(product_name, regex=False)
    
    # Boucle sur les termes séparés pour s'assurer que tous les termes sont présents dans le nom du produit.
    for term in product_name.split():
        mask &= open_food_dataframe['product_name'].str.contains(term, regex=False)
        
    # Extraction des informations sur le produit trouvé à partir de la base de données.
    product_info = open_food_dataframe[mask]
    
    # Limitation à 10 résultats
    if(len(product_info['product_name']) >= 10):
        var_nbr = 10
        product_info = product_info.head(10)
    else:
        var_nbr = len(product_info['product_name'])
        product_info = product_info.head(var_nbr)
    
    
    # Affichage du nombre de produits trouvés.
    if (len(product_info['product_name']) == 0):
        print("Aucun produit trouvé")
    else:
        with out:
            fig2 = px.bar(product_info, x='product_name', y='energy_100g', labels={'product_name':product_name, 'energy_100g': 'Valeur énergétique (en %)'})
            fig2.update_layout(yaxis_type="linear", yaxis_title="Valeur énergétique (en %)")
            # fig.show()
            display(fig2)
            
            fig = make_subplots(rows=var_nbr, cols=3, subplot_titles=list(product_info["product_name"]))
            
            for i in range(var_nbr):
                if ( e <= var_nbr ):
                    print(e)
                    
                    # le nombre de produits trouvés est affiché et les informations nutritionnelles sur les produits sont affichées sous forme de graphiques.
                    labels = ['Sugars', 'Fat', 'Carbohydrates', 'Proteins', 'Salt', 'nutrition_score_fr']
                    values = product_info.iloc[i][['sugars_100g', 'fat_100g', 'carbohydrates_100g', 'proteins_100g', 'salt_100g', 'nutrition_score_fr_100g']]
                    product_name = product_info.iloc[i]["product_name"]
                    
                    fig = plt.figure()
                    plt.bar(labels, values)
                    plt.xlabel('Nutrition')
                    plt.ylabel('Valeur')
                    plt.title(product_name)
                    plt.tight_layout()
                    
                    # Affichage
                    display(fig)
                    
                    fig3 = plt.figure()
                    plt.pie(values, labels=labels, autopct='%1.1f%%')
                    plt.title(product_name)

                    # Affichage
                    display(fig3)

                    # Mise à jour limitation du compteur de limitation à 10 éléments
                    e = e + 1
                else:
                    print("{} valeurs atteintes".format(var_nbr))

# Affichage de la zone de saisie de texte et du bouton
text = widgets.Text(description="Produit:")

# Bouton "Recherche".
def on_button_click(b):
    search(text.value)

button = widgets.Button(description="Rechercher")

button.on_click(on_button_click)

# Affichage des résultats de la recherche.
display(HBox([text, button]), out)


Output()